# Intermovie - projet 6 
Utilisation de dataframes pour manipuler des données. Exportation en CSV pour chaque dataframe créé. 

## Import des librairies

In [ ]:
import pandas as pd

%load_ext autoreload
%autoreload 2

## 2. Les notes moyennes des différents genres
### Import des datas initiaux :

In [ ]:
title_basics = pd.read_csv("../movies_dataset/title.basics.tsv",sep='\t', index_col='tconst', usecols=['tconst', 'genres', 'titleType'])
print('Title Basics imported.')

title_ratings = pd.read_csv("../movies_dataset/title.ratings.tsv",sep='\t', index_col='tconst', usecols=['tconst', 'averageRating'])
print('Title Ratings imported.')

### Nettoyage du dataset :

In [ ]:
title_basics = title_basics.dropna()
title_basics = title_basics[['genres']][title_basics.titleType.str.contains('movie|tvMovie', regex=True)]

### Jointure des tables :

In [27]:
# sauvegarde = title_basics.copy()
title_basics = sauvegarde.copy()

In [28]:
title_basics = title_basics.merge(title_ratings, how='left', left_index=True, right_index=True)
title_basics = title_basics.dropna()
title_basics = title_basics[title_basics.genres != '\\N']

In [29]:
moyenne_genre = pd.DataFrame(title_basics.genres.str.split(',').tolist(), index=title_basics.index).stack()
moyenne_genre = moyenne_genre.reset_index(['tconst', 0])

title_basics = title_basics.drop(columns=['genres'])

moyenne_genre = moyenne_genre.set_index('tconst')
moyenne_genre.columns = ['genre']

moyenne_genre = moyenne_genre.merge(title_basics, how='left', left_index=True, right_index=True)
moyenne_genre = moyenne_genre[moyenne_genre.genre != '\\N']

In [30]:
moyenne_genre = moyenne_genre.groupby('genre').mean()

### Import en CSV du fichier

In [31]:
moyenne_genre = moyenne_genre.sort_values(by=['averageRating'], ascending=False)

In [32]:
moyenne_genre.to_csv('moyenne_genre.csv', index=True)